In [56]:
# Importing Necessary libraries
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import torch.nn.functional as F
import numpy as np

In [47]:
# Transformation Pipelines

transformations = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

In [48]:
# Loading the train and test dataset
train_data = datasets.CIFAR10('cifar-10',train= True, transform= transformations, download= True)
test_data = datasets.CIFAR10('cifar-10',train= False, download= True)

Files already downloaded and verified
Files already downloaded and verified


In [49]:
len(train_data), len(test_data)

(50000, 10000)

In [50]:
# Creating a Validation Set from our training dataset

from torch.utils.data.sampler import SubsetRandomSampler

# Size of Validation Set
validation_size = 0.2

# Getting the size of training dataset
training_size = len(train_data)

# Creating the indices of training size
indices = list(range(training_size))

# Random Shuffling
np.random.shuffle(indices)

# Index Split
index_split = int(np.floor(training_size * validation_size))

# Setting Validation and Training Indices
validation_indices, training_indices = indices[:index_split], indices[index_split:]

# Sampling

training_sample = SubsetRandomSampler(training_indices)
validation_sample = SubsetRandomSampler(validation_indices)

In [51]:
# Creating DataLoader Object

from torch.utils.data.dataloader import DataLoader

batch_size = 16

trainloader = DataLoader(train_data, batch_size= batch_size, sampler= training_sample)
testloader = DataLoader(test_data, batch_size= batch_size)
validloader = DataLoader(train_data, batch_size= batch_size, sampler= validation_sample)

In [52]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        self.linear2 = nn.Linear(512, 10) 
        self.dropout = nn.Dropout(p=0.3)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [53]:
model = CNN()
print(model)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [54]:
# Checking if cuda is available or not

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device.type)

cpu


In [55]:
# Moving the model to the available device
model = model.to(device)

In [57]:
# Adding Loss Function

criterion = nn.CrossEntropyLoss()

# Adding Optimizer 

optimizer = optim.SGD(model.parameters(), lr= 0.01)

In [61]:
# Training Loop

n_epochs = 5

for epoch in range(1, n_epochs + 1):
  train_loss = 0.0
  valid_loss = 0.0

  model.train()

  for batch_idx, (data, target) in enumerate(trainloader):
    data, target = data.to(device) , target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    train_loss += loss.item() * data.size(0)

  model.eval()

  for batch_idx, (data, target) in enumerate(validloader):
    data, target = data.to(device), target.to(device)
    output = model(data)
    loss = criterion(output, target)
    valid_loss += loss.item() * data.size(0)

  train_loss = train_loss/len(trainloader.sampler)
  valid_loss = valid_loss/len(validloader.sampler)

  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f} |')

| Epoch: 01 | Train Loss: 1.000 | Val. Loss: 0.899 |
| Epoch: 02 | Train Loss: 0.976 | Val. Loss: 0.883 |
| Epoch: 03 | Train Loss: 0.956 | Val. Loss: 0.881 |
| Epoch: 04 | Train Loss: 0.934 | Val. Loss: 0.846 |
| Epoch: 05 | Train Loss: 0.918 | Val. Loss: 0.852 |
